# Lippmann-Schwinger equation in momentum space

We consider here the Lippmann-Schwinger equation in a fixed partial wave, written in momentum space as:

\begin{equation}
T(E_k+\mathrm{i}\varepsilon;p,k) = V(p,k)
+\int\frac{\mathrm{d}q\,q^2}{2\pi^2} V(p,q)
G_0(E_k+\mathrm{i}\varepsilon;q)
T(E_k+\mathrm{i}\varepsilon;q,k) \,,
\end{equation}

with $\varepsilon\to0$ implied.

## System and potential

Let us first define our system.  We study here the simplest possible case and set the mass $m=1$ as well as $\hbar c=1$.  This is actually the default for the `System` helper class we use:

In [ ]:
from lib.system import *

sys = System()

print(sys)

Now let us pick a potential.  We arbitrarily choose an attractive Gaussian potential with depth $V_0={-}4$ and range $R=2$.

In [ ]:
from lib.potential import *

V = V_Gauss(sys, -4.0, 2.0)

V.show()

## Mesh setup

Our first task is to discretize the momentum integral.  To that end we use a quadrature rule and replace

\begin{equation}
 \int \mathrm{d}q\,q^2 \longrightarrow \sum_j w_j p_j^2
\end{equation}

In [ ]:
from lib.mesh import *

While it is possible to define a quadrature rule on the infinite interval $[0,\infty)$, using a coordinate transformation that maps it onto a compact interval, in practice it is easier an to pick a sufficiently large
upper bound $p_{\text{max}}$.  For guidance how to choose this, we look at the potential in momentum space:

In [ ]:
V.show(rep='p', max=6.0)

This suggests that $p_{\text{max}}=4.0$ is sufficient.  We use this, and furthermore set the number of mesh points to 16.  **Both choices should later be varied to check that the calculation is sufficiently converged!**

In [ ]:
mesh = GaulegMesh(16, 0.0, 4.0)

## Green's function

The last ingredient we need is the free Green's function

\begin{equation}
 G_0(E;q) = \frac{1}{E-q^2}
\end{equation}

It is for convenience provided by our helper library, and it takes the `System` object to determine the proper conversion between energy and momentum.  In our case ($m=2\mu=1$), $E=E_k=k^2$.

In [ ]:
from lib.operator import *

G0 = G_0(sys)

Our $G_0(E_k,q)$ has a pole at q = k:

In [ ]:
import matplotlib.pyplot as plt

k = 1.0

plt.plot(mesh.ps(), G0(sys.e_from_k(k), mesh.ps()), marker="o")
plt.show()

The residue at the pole is also provided by the class:

In [ ]:
print(G0.residue(k))

This we will need to carry out the principal-value integration.

## Kernel matrix construction

We can now set up a matrix-vector equation that represents the discretize Lippmann-Schwinger equation, starting with the *kernel matrix*:

\begin{equation}
 K_{ij}(E_k) = \frac{w_jp_j^2}{2\pi^2} V(p_i,p_j) G_0(E_k;p_j)
\end{equation}

We have here replaced the integral by the quadrature, and chose the momentum $p$ as a point (indexed by $i$) on the same mesh.

Note that the energy $E_k$ is still a free parameter at this point, with momentum $k$ not associated with the mesh.  However, we need to pick $k$ and prepare the mesh for the principal value integration.  This is done as follows:

In [ ]:
k = 1.0

mesh.push_pv(k)

print(mesh.ps())
print(mesh.ws())

mesh.pop_pv()

We can see that the extra point and weight are added at the *end* of the mesh, but that is an *implementation detail* that a user of the `Mesh` class should not need to know about.  To realize that, there is a method `Mesh.psw()` that returns triples, with the first two entries giving the point and weight, and the third entry being a Boolean that indicates whether or not the point represents a pole contribution:

In [ ]:
mesh.push_pv(k)

for pw in mesh.pws():
  print(pw)
  
mesh.pop_pv()

An advantage of this, and the `push_pv()`/`pop_pv()` interface overall, is that the mesh class in principle supports an aribitrary number of pole terms.

**Note that after each usage the mesh should be restored to its default state by calling `pop_pv()`!**

We can now use this to construct the matrix.  Instead of manually looping to write rows and columns, we use `map()` here in the spirit of functional programming:

In [ ]:
def kernel(E):
  factor = 0.5 / np.pi**2
                                                                                     
  K = map(
    lambda p: list(map(
      lambda qw: factor * qw[1] * qw[0]**2 \
        * (G0.residue(qw[0]) if qw[2] else G0(E, qw[0])) \
        * V.get(0, p, qw[0]),
      mesh.pws()
    )), mesh.ps()
  )

  return np.asarray(list(K))  

In [ ]:
mesh.push_pv(k)
print(kernel(sys.e_from_k(k))[:, 0:3])
mesh.pop_pv()

## Solving the equation

The inhomogeneous term in the Lippmann-Schwinger equation, $V(p,k)$ is trivially discretized by setting $p=p_i$, as done for the construction of $K_{ij}$.  Schematically, we then obtain an equation of the form

\begin{equation}
 T_i = V_i + \sum_j K_{ij} T_j \,,
\end{equation}

which can be brought into the standard form for a linear equation system as follows:

\begin{equation}
 \sum_j (\delta_{ij} - K_{ij}) T_j  = V_i \,.
\end{equation}

It his this form that we solve numerically:

In [ ]:
def solve(k):
  mesh.push_pv(k)
  
  mat = np.identity(mesh.size()) - kernel(sys.e_from_k(k))
  vec = np.asarray(list(map(lambda p: V.get(0, p, k), mesh.ps())))
  
  sol = np.linalg.solve(mat, vec)
  
  mesh.pop_pv()
  
  return sol;

In [ ]:
print(solve(k))

The result is a whole vector, representing the *half off-shell T-matrix* on our mesh.  From the implementation of the principal-value integration we know again that the *on-shell value* is the last entry in this vector, but again we strive not to rely on such details.

One way to avoid this would be using the Lippmann-Schwinger equation with the solution vector $T_i$ inserted to interpolate the value for the on-shell point.  However, here we pick a simpler alternative and exploit that `push_pv()` actually returns the index of the newly added point:

In [ ]:
def solve_on_shell(k):
  i0 = mesh.push_pv(k)
  
  mat = np.identity(mesh.size()) - kernel(sys.e_from_k(k))
  vec = np.asarray(list(map(lambda p: V.get(0, p, k), mesh.ps())))
  
  sol = np.linalg.solve(mat, vec)
  
  mesh.pop_pv()
  
  return sol[i0];

In [ ]:
T = solve_on_shell(k)

print(T)

We can verify that this has the expected form $T = {-}\dfrac{2\pi}{\mu}\dfrac{1}{k\cot\delta(k)-\mathrm{i}k}$:

In [ ]:
print(-2.0 * np.pi / (k * sys.mu) * (1.0 / T))

And we can also calculate the phase shift:

In [ ]:
cot_delta = -2.0 * np.pi / (k * sys.mu) * (1.0 / T) + 1j
delta = np.arctan(1.0 / cot_delta).real
                 
print("delta = %g degrees" % np.rad2deg(delta))